# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.Connection('data.sqlite')
c = conn.cursor()

## Select the Average Number of Orders on a Product by Product Basis

In [3]:
c.execute("""select AVG(numOrders)
                    from (select COUNT(*) as numOrders FROM orderDetails GROUP BY productCode);
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,AVG(numOrders)
0,27.486239


## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

<img src="images/Database-Schema.png" width="600">

In [8]:
c.execute("""select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';"""
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [9]:
c.execute("""SELECT customerNumber, contactLastName, contactFirstName
            FROM customers WHERE customerNumber IN (SELECT customerNumber FROM orders WHERE orderDate = '2003-01-31')
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

<img src="images/Database-Schema.png" width="600">

In [21]:
c.execute("""SELECT productName, COUNT(*) as totalOrders, SUM(quantityOrdered) as totalQuantityOrdered
             FROM orderDetails
             JOIN products USING(productCode)
             GROUP BY productName
             ORDER BY totalQuantityOrdered DESC
"""
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,productName,totalOrders,totalQuantityOrdered
0,1992 Ferrari 360 Spider red,53,1808
1,1937 Lincoln Berline,28,1111
2,American Airlines: MD-11S,28,1085
3,1941 Chevrolet Special Deluxe Cabriolet,28,1076
4,1930 Buick Marquette Phaeton,28,1074


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

<img src="images/Database-Schema.png" width="600">

In [24]:
c.execute("""SELECT productName, COUNT(*) as peopleOrdered
             FROM orders
             JOIN orderdetails USING(orderNumber)
             JOIN customers USING(customerNumber)
             JOIN products USING(productCode)
             GROUP BY customerNumber
"""
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,productName,peopleOrdered
0,1938 Cadillac V-16 Presidential Limousine,7
1,1936 Mercedes Benz 500k Roadster,29
2,1940s Ford truck,55
3,Diamond T620 Semi-Skirted Tanker,53
4,1962 City of Detroit Streetcar,32
5,1938 Cadillac V-16 Presidential Limousine,180
6,1940s Ford truck,22
7,1996 Peterbilt 379 Stake Bed with Outrigger,21
8,Boeing X-32A JSF,49
9,1982 Camaro Z28,259


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [ ]:
#Your code here

## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [ ]:
#Your code here

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!